# 📄 이미지 PDF Azure Document Intelligence OCR + 레이아웃(표) 추출 파이프라인

목적: 이미지 기반 PDF를 Azure Document Intelligence(구 Form Recognizer)로 분석하여 페이지별 텍스트와 표를 추출하고, 읽기 순서를 컬럼(좌→우) 기준으로 복원합니다.

핵심 기능:
- /home/admin/wkms-aws/backend/.env 에서 엔드포인트/키 또는 AAD 자격정보 로드
- PDF를 직접 Azure에 분석 요청(이미지 변환 불필요), 필요 시 페이지 이미지 저장은 pdf2image 사용
- N-컬럼(2열 포함) 자동 감지/정렬로 좌→우 컬럼 순서 병합
- 전체 텍스트 파일, 페이지별 텍스트/이미지, 표 텍스트, 메타데이터(JSON) 저장

In [1]:
# ✅ 의존성 점검
import importlib

def check_lib(name):
    try:
        importlib.import_module(name)
        print(f'✔ {name} 사용 가능', flush=True)
        return True
    except Exception as e:
        print(f'✖ {name} 불가: {e}', flush=True)
        return False

needed = [
    'azure.ai.documentintelligence',  # 최신 SDK (있으면 우선)
    'azure.ai.formrecognizer',        # 구 SDK (fallback)
    'azure.core.credentials',
    'pdf2image',                      # 선택: 페이지 이미지 저장
    'PIL',
    'dotenv'
]
print('🔎 라이브러리 상태 확인:', flush=True)
status = {n: check_lib(n) for n in needed}
missing = [n for n, ok in status.items() if not ok]
if missing:
    print(f'⚠ 설치 필요: {", ".join(missing)}', flush=True)
else:
    print('✅ 모든 필수 라이브러리 확인됨', flush=True)

🔎 라이브러리 상태 확인:
✔ azure.ai.documentintelligence 사용 가능
✔ azure.ai.formrecognizer 사용 가능
✔ azure.core.credentials 사용 가능
✖ pdf2image 불가: No module named 'pdf2image'
✔ PIL 사용 가능
✔ dotenv 사용 가능
⚠ 설치 필요: pdf2image


In [3]:
# ⚙️ 경로/설정 및 Azure DI 초기화 (.env 로드)
import os, io, json, time, sys
from pathlib import Path
from dataclasses import dataclass, field
from typing import Optional, Dict, Any, List
from dotenv import load_dotenv

# ------------------------------------------------------------------
# 프로젝트/노트북 경로 동적 탐색 (하드코딩 제거)
# ------------------------------------------------------------------
THIS_NOTEBOOK_DIR = Path.cwd()

# 최상위 프로젝트 루트 추정 (backend 디렉터리가 있는 상위)
def find_project_root(start: Path) -> Path:
    cur = start
    for _ in range(10):  # 최대 10 단계 상위 탐색
        if (cur / 'backend').is_dir():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start  # 실패 시 현재 디렉터리

PROJECT_ROOT = find_project_root(THIS_NOTEBOOK_DIR)
JUPYTER_DIR = PROJECT_ROOT / 'jupyter_notebook'
DATA_DIR = JUPYTER_DIR / 'data'
INPUT_PDF = DATA_DIR / 'input_docs' / 'test.pdf'   # 요청된 테스트 파일
OUTPUT_DIR = JUPYTER_DIR / 'data' / 'azure_di_output' / 'extracted_texts'
ENV_PATH_CANDIDATES = [
    PROJECT_ROOT / 'backend' / '.env',                  # 백엔드 .env (우선)
    PROJECT_ROOT / '.env',                              # 루트 .env
]

# 출력 디렉토리 구조
@dataclass
class Paths:
    output_root: Path = OUTPUT_DIR
    page_texts: Path = field(default_factory=lambda: OUTPUT_DIR)
    page_images: Path = field(default_factory=lambda: OUTPUT_DIR / 'page_images')
    page_tables: Path = field(default_factory=lambda: OUTPUT_DIR / 'page_tables')

paths = Paths()
for d in [paths.output_root, paths.page_images, paths.page_tables]:
    d.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------------
# .env 로드 (첫 번째 존재하는 경로)
# ------------------------------------------------------------------
loaded_env = None
for p in ENV_PATH_CANDIDATES:
    if p.exists():
        try:
            load_dotenv(p, override=False)
            loaded_env = p
            print(f'✅ .env 로드: {p}', flush=True)
            break
        except Exception as e:
            print(f'⚠ .env 로드 실패({p}): {e}', flush=True)
if not loaded_env:
    print('⚠ 로드 가능한 .env 파일을 찾지 못했습니다 (환경변수 직접 사용).', flush=True)

# ------------------------------------------------------------------
# 환경 변수 정규화 & 호환 키 매핑
# ------------------------------------------------------------------
ENV = os.environ

def first_nonempty(*keys: str) -> Optional[str]:
    for k in keys:
        v = ENV.get(k)
        if v:
            return v
    return None

AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = first_nonempty(
    'AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT',
    'AZURE_DOCUMENTINTELLIGENCE_ENDPOINT',
    'AZURE_FORMRECOGNIZER_ENDPOINT'
)
AZURE_DOCUMENT_INTELLIGENCE_KEY = first_nonempty(
    'AZURE_DOCUMENT_INTELLIGENCE_API_KEY',
    'AZURE_DOCUMENTINTELLIGENCE_API_KEY',
    'AZURE_FORMRECOGNIZER_API_KEY'
)

# 모델: prebuilt-document → prebuilt-read → prebuilt-layout 순 폴백
AZURE_DOC_MODEL = first_nonempty('AZURE_DOCUMENT_INTELLIGENCE_DEFAULT_MODEL') or 'prebuilt-document'
if AZURE_DOC_MODEL not in {'prebuilt-document', 'prebuilt-read', 'prebuilt-layout'}:
    AZURE_DOC_MODEL = 'prebuilt-document'

CONFIDENCE_THRESHOLD = float(ENV.get('AZURE_DOCUMENT_INTELLIGENCE_CONFIDENCE_THRESHOLD', '0.75'))
MAX_PAGES = int(ENV.get('AZURE_DOCUMENT_INTELLIGENCE_MAX_ASYNC_PAGES', ENV.get('AZURE_DOCUMENT_INTELLIGENCE_MAX_PAGES', '50')))

AZURE_TENANT_ID = ENV.get('AZURE_TENANT_ID')
AZURE_CLIENT_ID = ENV.get('AZURE_CLIENT_ID')
AZURE_CLIENT_SECRET = ENV.get('AZURE_CLIENT_SECRET')

# ------------------------------------------------------------------
# SDK 임포트: formrecognizer 우선, 실패 시 documentintelligence
# ------------------------------------------------------------------
_Client = None
AzureKeyCredential = None
client = None
preferred_sdk = 'none'

try:
    from azure.ai.formrecognizer import DocumentAnalysisClient as _Client  # type: ignore
    from azure.core.credentials import AzureKeyCredential  # type: ignore
    preferred_sdk = 'formrecognizer'
except Exception:
    try:
        from azure.ai.documentintelligence import DocumentIntelligenceClient as _Client  # type: ignore
        from azure.core.credentials import AzureKeyCredential  # type: ignore
        preferred_sdk = 'documentintelligence'
    except Exception:
        _Client = None
        preferred_sdk = 'none'

if _Client is None:
    raise RuntimeError('Azure Document Intelligence/Form Recognizer SDK가 설치되어 있지 않습니다. pip install azure-ai-formrecognizer')

# ------------------------------------------------------------------
# 클라이언트 생성 (Key 우선, 없으면 AAD) + 엔드포인트 정리
# ------------------------------------------------------------------
if AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT:
    AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT.rstrip('/')

if AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT and AZURE_DOCUMENT_INTELLIGENCE_KEY:
    client = _Client(
        endpoint=AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT,
        credential=AzureKeyCredential(AZURE_DOCUMENT_INTELLIGENCE_KEY)
    )
elif (AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT and AZURE_TENANT_ID and AZURE_CLIENT_ID and AZURE_CLIENT_SECRET):
    try:
        from azure.identity import ClientSecretCredential  # type: ignore
        cred = ClientSecretCredential(
            tenant_id=AZURE_TENANT_ID,
            client_id=AZURE_CLIENT_ID,
            client_secret=AZURE_CLIENT_SECRET
        )
        client = _Client(endpoint=AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT, credential=cred)
    except Exception as e:
        raise RuntimeError(f'AAD 자격증명 생성 실패: {e}')
else:
    raise RuntimeError('엔드포인트/자격증명 설정이 부족합니다. AZURE_* 환경변수를 확인하세요.')

# ------------------------------------------------------------------
# 파이프라인 설정 구조체
# ------------------------------------------------------------------
@dataclass
class DIConfig:
    model_read: str = AZURE_DOC_MODEL  # 기본 모델 (prebuilt-document 권장)
    model_layout: str = 'prebuilt-layout'
    save_page_images: bool = True
    ocr_dpi: int = 220  # 이미지 저장 시에만 사용
    use_tables: bool = True
    # 컬럼 처리
    column_mode: str = 'auto'   # 'auto' | 'single' | 'two' | 'n'
    max_columns: int = 3
    min_lines_per_column: int = 3
    # 추가 설정
    confidence_threshold: float = CONFIDENCE_THRESHOLD
    max_pages: int = MAX_PAGES

di_cfg = DIConfig()

# ------------------------------------------------------------------
# 요약 출력 (진단 강화)
# ------------------------------------------------------------------
print('📂 출력 경로 설정 완료:', flush=True)
print(f' - 프로젝트 루트: {PROJECT_ROOT}', flush=True)
print(f' - 노트북 경로: {THIS_NOTEBOOK_DIR}', flush=True)
print(f' - 입력 PDF: {INPUT_PDF} (exists={INPUT_PDF.exists()})', flush=True)
print(f' - 페이지 텍스트: {paths.page_texts}', flush=True)
print(f' - 페이지 이미지: {paths.page_images}', flush=True)
print(f' - 표 출력: {paths.page_tables}', flush=True)
print('🔐 Azure 설정:', flush=True)
print(f' - SDK 선택: {preferred_sdk}', flush=True)
print(f' - ENDPOINT 제공 여부: {bool(AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT)}', flush=True)
if AZURE_DOCUMENT_INTELLIGENCE_KEY:
    masked = AZURE_DOCUMENT_INTELLIGENCE_KEY[:8] + '*'*8 + AZURE_DOCUMENT_INTELLIGENCE_KEY[-4:]
    print(f' - KEY: 설정됨 ({masked})', flush=True)
else:
    print(' - KEY: 미설정 (AAD 또는 실패)', flush=True)
print('🧭 컬럼/모델 설정:', flush=True)
print(f' - model_read={di_cfg.model_read}, model_layout={di_cfg.model_layout}', flush=True)
print(f' - mode={di_cfg.column_mode}, max_cols={di_cfg.max_columns}, min_lines/col={di_cfg.min_lines_per_column}', flush=True)
print(f' - confidence={di_cfg.confidence_threshold}, max_pages={di_cfg.max_pages}', flush=True)

# ------------------------------------------------------------------
# 사전 경고: 흔한 404 점검
# ------------------------------------------------------------------
if preferred_sdk == 'formrecognizer' and di_cfg.model_read == 'prebuilt-read':
    print('ℹ 참고: 일부 지역/신규 리소스에서는 prebuilt-read 대신 prebuilt-document 권장', flush=True)
if preferred_sdk == 'formrecognizer' and di_cfg.model_read == 'prebuilt-document':
    print('✅ prebuilt-document 모델 사용 (권장)', flush=True)

# NOTE: 이후 셀에서 client / di_cfg / paths / INPUT_PDF 사용.


✅ .env 로드: /home/wjadmin/Dev/InsightBridge/backend/.env
📂 출력 경로 설정 완료:
 - 프로젝트 루트: /home/wjadmin/Dev/InsightBridge
 - 노트북 경로: /home/wjadmin/Dev/InsightBridge/jupyter_notebook
 - 입력 PDF: /home/wjadmin/Dev/InsightBridge/jupyter_notebook/data/input_docs/test.pdf (exists=True)
 - 페이지 텍스트: /home/wjadmin/Dev/InsightBridge/jupyter_notebook/data/azure_di_output/extracted_texts
 - 페이지 이미지: /home/wjadmin/Dev/InsightBridge/jupyter_notebook/data/azure_di_output/extracted_texts/page_images
 - 표 출력: /home/wjadmin/Dev/InsightBridge/jupyter_notebook/data/azure_di_output/extracted_texts/page_tables
🔐 Azure 설정:
 - SDK 선택: formrecognizer
 - ENDPOINT 제공 여부: True
 - KEY: 설정됨 (3v1uuPY5********Edm3)
🧭 컬럼/모델 설정:
 - model_read=prebuilt-read, model_layout=prebuilt-layout
📂 출력 경로 설정 완료:
 - 프로젝트 루트: /home/wjadmin/Dev/InsightBridge
 - 노트북 경로: /home/wjadmin/Dev/InsightBridge/jupyter_notebook
 - 입력 PDF: /home/wjadmin/Dev/InsightBridge/jupyter_notebook/data/input_docs/test.pdf (exists=True)
 - 페이지 텍스트: /home/wjadmin/

In [4]:
# 🔧 유틸리티
def write_text(path: Path, content: str):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w', encoding='utf-8') as f:
        f.write(content)

def write_json(path: Path, data: Dict[str, Any]):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def is_image_pdf(pdf_path: Path, sample_pages: int = 3) -> bool:
    try:
        import pdfplumber  # type: ignore
        total_txt = ''
        with pdfplumber.open(str(pdf_path)) as pdf:
            for i, page in enumerate(pdf.pages[:sample_pages], 1):
                try:
                    total_txt += (page.extract_text() or '')
                except Exception:
                    pass
        return len((total_txt or '').strip()) < 100
    except Exception:
        return True  # 라이브러리 없거나 실패 시 보수적으로 이미지 PDF 간주

In [5]:
# 🧮 컬럼 감지/정렬 유틸(1D k-means + 정렬)
from typing import Tuple

def _kmeans_1d(values: List[float], k: int, iters: int = 15) -> List[int]:
    if k <= 1 or len(values) <= k:
        return [0 for _ in values]
    sorted_vals = sorted(set(values))
    if len(sorted_vals) < k:
        k = len(sorted_vals)
    centers = [sorted_vals[max(0, min(len(sorted_vals)-1, round((i+0.5)*len(sorted_vals)/k)-1))] for i in range(k)]
    assign = [0]*len(values)
    for _ in range(iters):
        for i, v in enumerate(values):
            ci = min(range(k), key=lambda j: abs(v - centers[j]))
            assign[i] = ci
        new_centers = centers[:]
        for j in range(k):
            group = [values[i] for i, a in enumerate(assign) if a == j]
            if group:
                new_centers[j] = sum(group)/len(group)
        if all(abs(new_centers[j]-centers[j]) < 1e-4 for j in range(k)):
            break
        centers = new_centers
    return assign

def _line_left_top_norm(line, page_w: float, page_h: float) -> Tuple[float, float]:
    poly = getattr(line, 'polygon', None) or getattr(line, 'bounding_polygon', None)
    if poly:
        xs = [getattr(p, 'x', p[0]) for p in poly]
        ys = [getattr(p, 'y', p[1]) for p in poly]
        left = (min(xs) / (page_w or 1.0))
        top = (min(ys) / (page_h or 1.0))
    else:
        left, top = 0.0, 0.0
    return left, top

def _split_into_n_columns(lines, page_w: float, page_h: float, max_cols: int, min_lines_per_col: int):
    if not lines:
        return [lines]
    xs = [_line_left_top_norm(l, page_w, page_h)[0] for l in lines]
    best_groups = [lines]
    for k in range(2, max_cols+1):
        assign = _kmeans_1d(xs, k)
        groups = [[] for _ in range(k)]
        for l, a in zip(lines, assign):
            groups[a].append(l)
        groups = [g for g in groups if len(g) >= min_lines_per_col]
        if len(groups) <= 1:
            continue
        def left_mean(g):
            vals = [_line_left_top_norm(x, page_w, page_h)[0] for x in g]
            return sum(vals)/len(vals)
        groups.sort(key=left_mean)
        for g in groups:
            g.sort(key=lambda b: (_line_left_top_norm(b, page_w, page_h)[1], _line_left_top_norm(b, page_w, page_h)[0]))
        best_groups = groups
    return best_groups

In [6]:
# 🔤 Azure DI OCR + 페이지별 저장 (컬럼 정렬 포함)
from typing import Dict, Any, List

def analyze_pdf_with_azure_di(pdf_path: Path, paths: Paths, cfg: DIConfig, client) -> Dict[str, Any]:
    # 페이지 이미지 저장(선택)용
    page_image_files: List[str] = []
    if cfg.save_page_images:
        try:
            from pdf2image import convert_from_path  # type: ignore
            images = convert_from_path(str(pdf_path), dpi=cfg.ocr_dpi)
            for idx, img in enumerate(images, 1):
                page_img_path = paths.page_images / f'{pdf_path.stem}_page_{idx:03d}.png'
                try:
                    img.save(page_img_path)
                    page_image_files.append(str(page_img_path))
                except Exception:
                    pass
        except Exception:
            pass

    with open(pdf_path, 'rb') as f:
        data = f.read()
    start = time.time()
    print('🔄 Azure Read 모델 분석 요청...', flush=True)
    poller = client.begin_analyze_document(cfg.model_read, document=data)
    result = poller.result()
    print('   ✅ Read 분석 완료', flush=True)

    full_text_parts: List[str] = []
    page_text_files: List[str] = []
    page_meta: List[Dict[str, Any]] = []

    # 페이지 단위로 라인 수집 및 컬럼 정렬 후 저장
    pages = getattr(result, 'pages', None) or []
    for page in pages:
        idx = int(getattr(page, 'page_number', None) or getattr(page, 'page', None) or 0)
        page_w = float(getattr(page, 'width', None) or 1.0)
        page_h = float(getattr(page, 'height', None) or 1.0)
        lines = list(getattr(page, 'lines', None) or [])
        # 컬럼 그룹 결정
        if cfg.column_mode == 'single':
            groups = [lines]
        elif cfg.column_mode == 'two':
            groups = _split_into_n_columns(lines, page_w, page_h, max_cols=2, min_lines_per_col=cfg.min_lines_per_column)
        elif cfg.column_mode == 'n':
            groups = _split_into_n_columns(lines, page_w, page_h, max_cols=cfg.max_columns, min_lines_per_col=cfg.min_lines_per_column)
        else:
            groups = _split_into_n_columns(lines, page_w, page_h, max_cols=2, min_lines_per_col=cfg.min_lines_per_column)
            if len(groups) <= 1 and cfg.max_columns > 2:
                groups = _split_into_n_columns(lines, page_w, page_h, max_cols=cfg.max_columns, min_lines_per_col=cfg.min_lines_per_column)
            if len(groups) <= 1:
                groups = [lines]
        # 그룹 내 정렬 보강
        for g in groups:
            g.sort(key=lambda b: (_line_left_top_norm(b, page_w, page_h)[1], _line_left_top_norm(b, page_w, page_h)[0]))
        # 텍스트 병합(좌 컬럼 전체 → 우 컬럼 전체)
        ordered_text = ['\n'.join([getattr(b, 'content', getattr(b, 'text', '')).strip() for b in g if getattr(b, 'content', getattr(b, 'text', ''))]).strip() for g in groups if g]
        txt = '\n\n'.join([t for t in ordered_text if t]).strip()
        page_txt_path = paths.page_texts / f'{pdf_path.stem}_page_{idx:03d}.txt'
        write_text(page_txt_path, txt)
        page_text_files.append(str(page_txt_path))
        full_text_parts.append(f'\n\n=== 페이지 {idx} ===\n' + txt)
        page_meta.append({
            'page': idx,
            'text_length': len(txt),
            'text_file': str(page_txt_path),
            'image_file': page_image_files[idx-1] if 0 < idx <= len(page_image_files) else None,
            'columns_detected': len(groups)
        })

    full_text = '\n'.join(full_text_parts).strip()
    full_txt_path = paths.output_root / f'{pdf_path.stem}_azuredi_full.txt'
    write_text(full_txt_path, full_text)

    meta: Dict[str, Any] = {
        'method': 'azure-di-read',
        'pages': len(pages),
        'total_time': round(time.time() - start, 3),
        'page_details': page_meta,
        'output_file': str(full_txt_path),
        'page_text_files': page_text_files,
        'page_image_files': page_image_files
    }
    return {'text': full_text, 'metadata': meta}

In [7]:
# 📊 Azure DI 표 추출(prebuilt-layout) → 텍스트 저장
def extract_tables_with_azure_layout(pdf_path: Path, paths: Paths, client, model_layout: str = 'prebuilt-layout') -> Dict[str, Any]:
    try:
        with open(pdf_path, 'rb') as f:
            data = f.read()
        print('📊 Azure Layout 모델 분석 요청(테이블)...', flush=True)
        poller = client.begin_analyze_document(model_layout, document=data)
        res = poller.result()
    except Exception as e:
        print(f'⚠ Azure Layout 분석 실패: {e}', flush=True)
        return {'tables': [], 'files': []}

    saved: List[str] = []
    tables_summary: List[Dict[str, Any]] = []
    tables = getattr(res, 'tables', None) or []
    for t_i, tbl in enumerate(tables, 1):
        # 페이지 번호 추정
        page_num = None
        try:
            # 일부 SDK에선 cells가 page_number를 가짐
            if getattr(tbl, 'cells', None):
                for c in tbl.cells:
                    page_num = getattr(c, 'page_number', None) or getattr(c, 'page', None)
                    if page_num:
                        break
        except Exception:
            pass
        cells = getattr(tbl, 'cells', None) or []
        max_row = max([getattr(c, 'row_index', 0) for c in cells] + [0])
        max_col = max([getattr(c, 'column_index', 0) for c in cells] + [0])
        grid = {r: {} for r in range(1, max_row + 2)}
        for c in cells:
            r = getattr(c, 'row_index', 0) + 1
            col = getattr(c, 'column_index', 0) + 1
            content = getattr(c, 'content', getattr(c, 'text', '')) or ''
            grid.setdefault(r, {})[col] = content.strip()
        lines = []
        for r in range(1, max_row + 2):
            row_vals = [grid.get(r, {}).get(c, '') for c in range(1, max_col + 2)]
            lines.append(' | '.join([v.strip() for v in row_vals]))
        header = f"=== 페이지 {page_num or '?'} 테이블 {t_i} ===\n"
        content = (header + '\n'.join(lines)).strip()
        out = paths.page_tables / f'{pdf_path.stem}_p{(int(page_num) if page_num else 0):03d}_t{t_i:02d}.txt'
        write_text(out, content)
        saved.append(str(out))
        tables_summary.append({'page': int(page_num or 0), 'table_index': t_i, 'rows': len(lines), 'cols': (max_col + 1)})
    return {'tables': tables_summary, 'files': saved}

In [8]:
# 🧠 오케스트레이터
def process_pdf_with_azure_di(pdf_path: Path, paths: Paths, cfg: DIConfig, client) -> Dict[str, Any]:
    print(f'📥 입력 파일: {pdf_path}', flush=True)
    if not pdf_path.exists():
        raise FileNotFoundError(f'파일이 없습니다: {pdf_path}')
    img_pdf = is_image_pdf(pdf_path)
    print(f'🧪 이미지 PDF 판별: {"예" if img_pdf else "아니오"}', flush=True)
    print(f'🧭 컬럼 모드: {cfg.column_mode}, max_cols={cfg.max_columns}, min_lines/col={cfg.min_lines_per_column}', flush=True)

    # 1) Read 기반 텍스트 추출(+컬럼 정렬)
    res = analyze_pdf_with_azure_di(pdf_path, paths, cfg, client)

    # 2) 표 추출(선택: layout)
    table_info = {'tables': [], 'files': []}
    if cfg.use_tables:
        table_info = extract_tables_with_azure_layout(pdf_path, paths, client, cfg.model_layout)

    # 3) 메타데이터 합성
    meta = res.get('metadata', {})
    meta.update({'tables': table_info.get('tables', []), 'table_files': table_info.get('files', [])})
    doc_meta_path = paths.output_root / f'{pdf_path.stem}_azuredi_metadata.json'
    write_json(doc_meta_path, meta)

    print('✅ 처리 완료', flush=True)
    print(f' - 전체 텍스트 파일: {meta.get("output_file")}', flush=True)
    print(f' - 표 파일 수: {len(meta.get("table_files", []))}', flush=True)
    print(f' - 페이지 텍스트 파일 수: {len(meta.get("page_text_files", []))}', flush=True)
    print(f' - 페이지 이미지 파일 수: {len(meta.get("page_image_files", []))}', flush=True)
    return {'text': res.get('text', ''), 'metadata': meta}

In [9]:
# ▶️ 실행 러너: test.pdf
pdf_path = INPUT_PDF
print('🚀 Azure DI OCR + 표 추출 데모 시작', flush=True)
print(f'입력: {pdf_path}', flush=True)
result = process_pdf_with_azure_di(pdf_path, paths, di_cfg, client)

# 상위 500자 미리보기
preview = (result.get('text') or '')[:500]
print('\n🔎 전체 텍스트 미리보기(상위 500자):', flush=True)
print(preview if preview else '(미리보기 없음)', flush=True)

🚀 Azure DI OCR + 표 추출 데모 시작
입력: /home/wjadmin/Dev/InsightBridge/jupyter_notebook/data/input_docs/test.pdf
📥 입력 파일: /home/wjadmin/Dev/InsightBridge/jupyter_notebook/data/input_docs/test.pdf
입력: /home/wjadmin/Dev/InsightBridge/jupyter_notebook/data/input_docs/test.pdf
📥 입력 파일: /home/wjadmin/Dev/InsightBridge/jupyter_notebook/data/input_docs/test.pdf
🧪 이미지 PDF 판별: 예
🧭 컬럼 모드: auto, max_cols=3, min_lines/col=3
🧪 이미지 PDF 판별: 예
🧭 컬럼 모드: auto, max_cols=3, min_lines/col=3
🔄 Azure Read 모델 분석 요청...
🔄 Azure Read 모델 분석 요청...
   ✅ Read 분석 완료
   ✅ Read 분석 완료
📊 Azure Layout 모델 분석 요청(테이블)...
📊 Azure Layout 모델 분석 요청(테이블)...
✅ 처리 완료
 - 전체 텍스트 파일: /home/wjadmin/Dev/InsightBridge/jupyter_notebook/data/azure_di_output/extracted_texts/test_azuredi_full.txt
 - 표 파일 수: 11
 - 페이지 텍스트 파일 수: 27
 - 페이지 이미지 파일 수: 0

🔎 전체 텍스트 미리보기(상위 500자):
=== 페이지 1 ===
韓國IT서비스學會誌
第22卷 第3號
6月, pp.1-27
2023年
양손잡이 리더십과 혁신적인 업무 행동:
한국 반도체 산업의 증거
돔페 헨리 아메요* · 오포리 헨리 *** · 윤소라 **** . 강주영 *****
Ambidextrous Leadership and Innovative Work

## 참고
- .env 키 참조(우선순위):
  - 엔드포인트: AZURE_DOCUMENTINTELLIGENCE_ENDPOINT 또는 AZURE_FORMRECOGNIZER_ENDPOINT
  - 키: AZURE_DOCUMENTINTELLIGENCE_API_KEY 또는 AZURE_FORMRECOGNIZER_API_KEY
  - (옵션) AAD: AZURE_TENANT_ID, AZURE_CLIENT_ID, AZURE_CLIENT_SECRET
- Azure는 PDF를 바로 분석할 수 있으므로 OCR을 위해 이미지 변환이 필수는 아닙니다. 본 노트북은 페이지 이미지를 시각 점검용으로 선택 저장합니다.
- 컬럼 자동 감지 로직은 휴리스틱입니다. 문서 유형에 따라 DIConfig의 column_mode/max_columns/min_lines_per_column을 조정하세요.
- 출력물:
  - 전체 텍스트 파일: <파일명>_azuredi_full.txt
  - 페이지별 텍스트 파일: <파일명>_page_###.txt
  - 페이지 이미지(옵션): page_images/ 폴더 내 PNG
  - 표 텍스트: page_tables/ 폴더 내 텍스트 파일
  - 문서 메타데이터 JSON: <파일명>_azuredi_metadata.json